In [1]:
pip install tensorflow scikit-learn pillow matplotlib

  Using cached tensorflow-2.19.0-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.1.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached grpcio-1.73.0-cp311-cp311-win_amd64.whl.metadata (4.0 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.10.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached h5py-3.14.0-cp311-cp311-win_amd64.whl.metadata (2.7 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-win_amd64.whl.metadata (22 kB)
  Using cached tens

In [2]:
import os
import cv2
import sys
import time
import json
import shutil
import random
import zipfile
import itertools
import kagglehub
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from tqdm import trange, tqdm
from google.colab import files
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report

# Set random seed for reproducibility
np.random.seed(42)

ImportError: Traceback (most recent call last):
  File "C:\Users\asus\anaconda3\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Une routine d’initialisation d’une bibliothèque de liens dynamiques (DLL) a échoué.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
# Activation Functions
def relu(x):
    """ReLU activation: max(0, x)"""
    assert isinstance(x, np.ndarray), "Input to ReLU must be a numpy array"
    result = np.maximum(0, x)
    assert np.all(result >= 0), "ReLU output must be non-negative"
    return result

def relu_derivative(x):
    """Derivative of ReLU: 1 if x > 0, else 0"""
    assert isinstance(x, np.ndarray), "Input to ReLU derivative must be a numpy array"
    result = np.where(x > 0, 1, 0)
    assert np.all((result == 0) | (result == 1)), "ReLU derivative must be 0 or 1"
    return result

def softmax(x):
    """Softmax activation: exp(x) / sum(exp(x))"""
    assert isinstance(x, np.ndarray), "Input to softmax must be a numpy array"
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    result = exp_x / np.sum(exp_x, axis=1, keepdims=True)
    assert np.all((result >= 0) & (result <= 1)), "Softmax output must be in [0,1]"
    assert np.allclose(np.sum(result, axis=1), 1), "Softmax output must sum to 1"
    return result

In [ ]:
def augment_image(image, max_rotation=10, max_translation=3):
    """Apply random rotation and translation to a 32x32 grayscale image"""
    assert image.shape == (32, 32), "Image must be 32x32"
    angle = np.random.uniform(-max_rotation, max_rotation)
    M = cv2.getRotationMatrix2D((16, 16), angle, 1.0)
    rotated = cv2.warpAffine(image, M, (32, 32), borderMode=cv2.BORDER_REPLICATE)
    tx = np.random.uniform(-max_translation, max_translation)
    ty = np.random.uniform(-max_translation, max_translation)
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    translated = cv2.warpAffine(rotated, M, (32, 32), borderMode=cv2.BORDER_REPLICATE)
    return translated.flatten()

# Image Preprocessing
def load_and_preprocess_image(image_path, data_dir, target_size=(32, 32)):
    """Load and preprocess image: grayscale, resize, normalize"""
    full_path = os.path.join(data_dir, image_path)
    assert os.path.exists(full_path), f"Image not found: {full_path}"
    img = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
    assert img is not None, f"Failed to load image: {full_path}"
    img = cv2.resize(img, target_size)
    img = img.astype(np.float32) / 255.0
    return img.flatten()

In [ ]:
def display_sample_images_from_paths(labels_df, data_dir, path_column='image_path', label_column='label',
                                     num_samples=33, samples_per_row=11, random_state=83):
    """Display a grid of sample grayscale images, one per class"""
    assert path_column in labels_df.columns, f"Missing column: {path_column}"
    assert label_column in labels_df.columns, f"Missing column: {label_column}"

    rng = np.random.default_rng(random_state)
    unique_labels = labels_df[label_column].unique()
    num_samples = min(num_samples, len(unique_labels))
    selected_indices = [rng.choice(labels_df[labels_df[label_column] == lbl].index)
                        for lbl in rng.choice(unique_labels, size=num_samples, replace=False)]
    selected_df = labels_df.loc[selected_indices].sort_values(by=label_column)

    num_rows = (num_samples + samples_per_row - 1) // samples_per_row
    fig, axes = plt.subplots(num_rows, samples_per_row, figsize=(samples_per_row * 2, num_rows * 3))
    axes = axes.ravel()

    for i, (path, label) in enumerate(zip(selected_df[path_column], selected_df[label_column])):
        try:
            img = Image.open(os.path.join(data_dir, path))
            axes[i].imshow(img, cmap='gray')
            axes[i].set_title(f"Label: {label}", fontsize=17)
            axes[i].axis('off')
        except Exception as e:
            print(f"Error loading {path}: {e}")
            axes[i].set_title("Error")
            axes[i].axis('off')

    for ax in axes[num_samples:]:
        ax.axis('off')

    plt.tight_layout()
    return figr

In [ ]:
class MulticlassNeuralNetwork:
    def __init__(self, layer_sizes, learning_rate=0.01, l2_lambda=0.01, optimizer='sgd', beta1=0.9, beta2=0.999, epsilon=1e-8):
        """Initialize MLP with specified architecture and parameters"""
        assert len(layer_sizes) >= 2, "At least 2 layers required"
        assert all(size > 0 for size in layer_sizes), "Layer sizes must be positive"
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.l2_lambda = l2_lambda
        self.optimizer = optimizer.lower()
        self.beta1, self.beta2, self.epsilon = beta1, beta2, epsilon
        self.weights = []
        self.biases = []
        self.m_weights, self.v_weights = [], []
        self.m_biases, self.v_biases = [], []
        self.t = 0

        # He initialization
        np.random.seed(42)
        for i in range(len(layer_sizes) - 1):
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i])
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)
            self.m_weights.append(np.zeros_like(w))
            self.v_weights.append(np.zeros_like(w))
            self.m_biases.append(np.zeros_like(b))
            self.v_biases.append(np.zeros_like(b))

    def forward(self, X):
        """Forward propagation"""
        self.z_values, self.activations = [], [X]
        for i in range(len(self.weights) - 1):
            Z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            self.z_values.append(Z)
            self.activations.append(relu(Z))
        Z = np.dot(self.activations[-1], self.weights[-1]) + self.biases[-1]
        self.z_values.append(Z)
        output = softmax(Z)
        self.activations.append(output)
        return output

    def compute_loss(self, y_true, y_pred):
        """Cross-entropy loss with L2 regularization"""
        m = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-15, 1 - 1e-15)
        cross_entropy = -np.sum(y_true * np.log(y_pred_clipped)) / m
        l2_term = (self.l2_lambda / (2 * m)) * sum(np.sum(np.square(W)) for W in self.weights)
        total_loss = cross_entropy + l2_term
        assert not np.isnan(total_loss), "Loss is NaN"
        return total_loss

    def compute_accuracy(self, y_true, y_pred):
        """Compute accuracy"""
        predictions = np.argmax(y_pred, axis=1)
        true_labels = np.argmax(y_true, axis=1)
        return np.mean(predictions == true_labels)

    def backward(self, X, y):
        """Backpropagation"""
        m = X.shape[0]
        self.d_weights = [np.zeros_like(w) for w in self.weights]
        self.d_biases = [np.zeros_like(b) for b in self.biases]
        dZ = self.activations[-1] - y
        self.d_weights[-1] = (np.dot(self.activations[-2].T, dZ) + self.l2_lambda * self.weights[-1]) / m
        self.d_biases[-1] = np.sum(dZ, axis=0, keepdims=True) / m
        dA_prev = dZ
        for i in range(len(self.weights) - 2, -1, -1):
            dA = np.dot(dA_prev, self.weights[i + 1].T)
            dZ = dA * relu_derivative(self.z_values[i])
            self.d_weights[i] = (np.dot(self.activations[i].T, dZ) + self.l2_lambda * self.weights[i]) / m
            self.d_biases[i] = np.sum(dZ, axis=0, keepdims=True) / m
            dA_prev = dZ

    def update_parameters(self):
        """Update parameters using SGD or Adam"""
        if self.optimizer == 'sgd':
            for i in range(len(self.weights)):
                self.weights[i] -= self.learning_rate * self.d_weights[i]
                self.biases[i] -= self.learning_rate * self.d_biases[i]
        elif self.optimizer == 'adam':
            self.t += 1
            for i in range(len(self.weights)):
                self.m_weights[i] = self.beta1 * self.m_weights[i] + (1 - self.beta1) * self.d_weights[i]
                self.v_weights[i] = self.beta2 * self.v_weights[i] + (1 - self.beta2) * (self.d_weights[i] ** 2)
                self.m_biases[i] = self.beta1 * self.m_biases[i] + (1 - self.beta1) * self.d_biases[i]
                self.v_biases[i] = self.beta2 * self.v_biases[i] + (1 - self.beta2) * (self.d_biases[i] ** 2)
                m_w_hat = self.m_weights[i] / (1 - self.beta1 ** self.t)
                v_w_hat = self.v_weights[i] / (1 - self.beta2 ** self.t)
                m_b_hat = self.m_biases[i] / (1 - self.beta1 ** self.t)
                v_b_hat = self.v_biases[i] / (1 - self.beta2 ** self.t)
                self.weights[i] -= self.learning_rate * m_w_hat / (np.sqrt(v_w_hat) + self.epsilon)
                self.biases[i] -= self.learning_rate * m_b_hat / (np.sqrt(v_b_hat) + self.epsilon)

    def train(self, X, y, X_val, y_val, epochs=100, batch_size=100, augment=False, early_stopping=True, verbose=1):
        """Train the MLP"""
        train_losses, val_losses, train_accuracies, val_accuracies = [], [], [], []
        best_val_loss = np.inf
        best_weights = [w.copy() for w in self.weights]
        best_biases = [b.copy() for b in self.biases]
        patience, patience_counter = 10, 0

        for epoch in range(epochs):
            start_time = time.time()

            indices = np.random.permutation(X.shape[0])
            X_shuffled, y_shuffled = X[indices], y[indices]
            epoch_loss = 0
            for i in range(0, X.shape[0], batch_size):
                X_batch = X_shuffled[i:i + batch_size]
                y_batch = y_shuffled[i:i + batch_size]
                if augment:
                    X_batch = np.array([augment_image(img.reshape(32, 32)) for img in X_batch])
                outputs = self.forward(X_batch)
                epoch_loss += self.compute_loss(y_batch, outputs)
                self.backward(X_batch, y_batch)
                self.update_parameters()

            train_pred = self.forward(X)
            val_pred = self.forward(X_val)
            train_loss = epoch_loss / (X.shape[0] // batch_size)
            val_loss = self.compute_loss(y_val, val_pred)
            train_accuracy = self.compute_accuracy(y, train_pred)
            val_accuracy = self.compute_accuracy(y_val, val_pred)

            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_accuracy)
            val_accuracies.append(val_accuracy)

            end_time = time.time()
            epoch_time = end_time - start_time

            if early_stopping:
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_weights = [w.copy() for w in self.weights]
                    best_biases = [b.copy() for b in self.biases]
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        if verbose >= 1:
                            print(f"Early stopping at epoch {epoch + 1}")
                        self.weights = best_weights
                        self.biases = best_biases
                        break

            log = (epoch % 10 == 0 or epoch == epochs - 1)
            if verbose == 2 and log:
                print(f"Epoch {epoch + 1:03d} | Time: {epoch_time:.2f}s | "
                      f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
                      f"Train Acc: {train_accuracy:.4f} | Val Acc: {val_accuracy:.4f}")
            elif verbose == 1 and log:
                print(f"\nEpoch {epoch + 1:03d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
                      f"Train Acc: {train_accuracy:.4f} | Val Acc: {val_accuracy:.4f}")

        return train_losses, val_losses, train_accuracies, val_accuracies



    def predict(self, X):
        """Predict class labels"""
        outputs = self.forward(X)
        return np.argmax(outputs, axis=1)

In [ ]:
def k_fold_cross_validation(X, y_one_hot, layer_sizes, k=5, epochs=100, batch_size=32,
                            learning_rate=0.001, l2_lambda=0.01, optimizer='sgd',
                            augment=False, early_stopping=False, verbose=1):
    assert isinstance(X, np.ndarray) and isinstance(y_one_hot, np.ndarray), "X and y_one_hot must be numpy arrays"
    assert X.shape[0] == y_one_hot.shape[0], "X and y_one_hot must have the same number of samples"
    assert isinstance(layer_sizes, list) and len(layer_sizes) >= 2, "layer_sizes must be a list with at least 2 elements"
    assert isinstance(k, int) and k > 1, "k must be an integer greater than 1"
    assert isinstance(epochs, int) and epochs > 0, "Epochs must be a positive integer"
    assert isinstance(batch_size, int) and batch_size > 0, "Batch size must be a positive integer"
    assert optimizer.lower() in ['sgd', 'adam'], "Optimizer must be 'sgd' or 'adam'"

    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_train_losses, fold_val_losses = [], []
    fold_train_accuracies, fold_val_accuracies = [], []

    all_splits = list(kf.split(X))

    for fold in trange(k, desc="Cross-validation folds", disable=(verbose == 0)):
        train_idx, val_idx = all_splits[fold]

        if verbose >= 1:
            print(f"\n=========================== Fold {fold + 1}/{k} ===========================================\n")
        start_time = time.time()

        X_train, X_val = X[train_idx], X[val_idx]
        y_train_one_hot, y_val_one_hot = y_one_hot[train_idx], y_one_hot[val_idx]

        nn = MulticlassNeuralNetwork(
            layer_sizes, learning_rate=learning_rate, l2_lambda=l2_lambda, optimizer=optimizer
        )

        train_losses, val_losses, train_accuracies, val_accuracies = nn.train(
            X_train, y_train_one_hot, X_val, y_val_one_hot,
            epochs=epochs, batch_size=batch_size,
            augment=augment, early_stopping=early_stopping, verbose=verbose
        )

        fold_train_losses.append(train_losses)
        fold_val_losses.append(val_losses)
        fold_train_accuracies.append(train_accuracies)
        fold_val_accuracies.append(val_accuracies)

        final_val_accuracy = val_accuracies[-1]
        if verbose >= 2:
            y_val_pred = nn.predict(X_val)
            print(f"\n===========================Fold {fold + 1} Classification Report: =======================")
            print(classification_report(np.argmax(y_val_one_hot, axis=1), y_val_pred))

        if verbose >= 1:
            print(f"Fold {fold + 1} completed in {time.time() - start_time:.2f} seconds")
            print(f"Final validation accuracy: {final_val_accuracy:.4f}")

    avg_train_losses = np.mean(fold_train_losses, axis=0)
    avg_val_losses = np.mean(fold_val_losses, axis=0)
    avg_train_accuracies = np.mean(fold_train_accuracies, axis=0)
    avg_val_accuracies = np.mean(fold_val_accuracies, axis=0)

    if verbose >= 1:
        print(f"\n=========================== Cross-Validation Summary ===========================")
        print(f"Average final validation accuracy: {avg_val_accuracies[-1]:.4f}")
        print(f"Metrics computed over {epochs} epochs")

    return avg_train_losses, avg_val_losses, avg_train_accuracies, avg_val_accuracies

In [ ]:
path = kagglehub.dataset_download("benaddym/amazigh-handwritten-character-database-amhcd")

target_dir = "/content/amhcd"
cleanup(temp_dir=target_dir)
shutil.copytree(path, target_dir, dirs_exist_ok=True)

print("Path to dataset files:", target_dir)

In [ ]:
latin_labels_txt    = target_dir + '/labels/labels/33-common-latin-tifinagh.txt'

tifinagh_labels_txt = target_dir + '/labels/labels/sorted-33-common-tifinagh.txt'
images_root         = target_dir + '/AMHCD_64/AMHCD_64'
output_csv          = target_dir + '/rgb-labels-map.csv'

# Lire les labels Latin et les glyphes Tifinagh
with open(latin_labels_txt, 'r', encoding='utf-8') as f:
    ascii_labels = [line.strip() for line in f if line.strip()]

with open(tifinagh_labels_txt, 'r', encoding='utf-8') as f:
    tif_chars = [line.strip() for line in f if line.strip()]

#  Vérification
if len(ascii_labels) != len(tif_chars):
    print(f"Erreur : {len(ascii_labels)} labels Latin ≠ {len(tif_chars)} glyphes Tifinagh")
    sys.exit(1)

In [ ]:
with open(output_csv, 'w', encoding='utf-8', newline='') as out:
    for latin, tif in zip(ascii_labels, tif_chars):
        folder = os.path.join(images_root, latin)
        if not os.path.isdir(folder):
            print(f"Dossier introuvable, je passe : {folder}")
            continue
        for fname in sorted(os.listdir(folder)):
            # filtrer si nécessaire par extension d’image
            if not fname.lower().endswith(('.png','.jpg','.jpeg','.bmp','.tif','.tiff')):
                continue
            rel_path = f"{images_root}/{latin}/{fname}"
            # Écrire UNE ligne contenant chemin + glyphe
            out.write(f"{rel_path},{tif}\n")

print(f"Généré {output_csv} avec {sum(len(os.listdir(os.path.join(images_root,l))) for l in ascii_labels if os.path.isdir(os.path.join(images_root,l)))} entrées.")

In [ ]:
labels_df = pd.read_csv(output_csv, names=['image_path', 'label'])

In [ ]:
print("Sample image paths:")
for path in labels_df['image_path'].head(5):
    full_path = os.path.join(target_dir, path)
    print(f"{full_path} -> Exists: {os.path.exists(full_path)}")

print(f"\nLoaded {len(labels_df)} samples with {labels_df['label'].nunique ()} unique classes .")

In [ ]:
class_counts = labels_df['label'].value_counts()
print("Class distribution summary:")
print(f"Min: {class_counts.min()}, Max: {class_counts.max()}, Mean: {class_counts.mean():.2f}")

In [ ]:
print("Label encoding ...")
# Encode labels
label_encoder = LabelEncoder()
labels_df['label_encoded'] = label_encoder.fit_transform(labels_df['label'])
num_classes = len(label_encoder.classes_)

print("==================== Data with label encoded: ======================")
print(labels_df)

In [ ]:
print(f"Class distribution:")
# display(labels_df['label'].value_counts())
display(labels_df[['label', 'label_encoded']].value_counts())

In [ ]:
X = np.array([load_and_preprocess_image(path, target_dir) for path in labels_df['image_path']])
y = labels_df['label_encoded'].values

# Verify dimensions
assert X.shape[0] == y.shape[0], "Mismatch between number of images and labels"
assert X.shape[1] == 32 * 32, f"Expected flattened image size of {32 * 32}, got {X.shape[1]}"

In [ ]:
fig = display_sample_images_from_paths(labels_df, target_dir, random_state=4)
plt.savefig('sample_images1.png')
files.download('sample_images1.png')
plt.close()

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42)

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train_one_hot = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_val_one_hot = one_hot_encoder.transform(y_val.reshape(-1, 1))
y_test_one_hot = one_hot_encoder.transform(y_test.reshape(-1, 1))
print(f"Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}")

In [ ]:
layer_sizes = [32 * 32, 64, 32, num_classes]
param_distributions = {
    'learning_rate': [0.01, 0.001],
    'l2_lambda': [0.01, 0.001],
    'batch_size': [64, 32],
    'optimizer': ['adam', 'sgd']
}

n_iter = 16

# All possible combinations
keys = list(param_distributions.keys())
all_combinations = [
    dict(zip(keys, values))
    for values in itertools.product(*param_distributions.values())
]

# Shuffle and pick n_iter unique combinations
random.shuffle(all_combinations)
random_param_combinations  = all_combinations[:n_iter]

assert len(random_param_combinations) == len(set(tuple(sorted(p.items())) for p in random_param_combinations)), "Duplicate combinations found!"

results = []
best_val_accuracy = 0
best_params = None

In [ ]:
for i in trange(n_iter, desc="Random Search"):
    params = random_param_combinations[i]
    print(f"\nTrial {i + 1}/{n_iter}: {params}")
    avg_train_losses, avg_val_losses, avg_train_accuracies, avg_val_accuracies = k_fold_cross_validation(
        X_train, y_train_one_hot, layer_sizes, k=5, epochs=100, **params, augment=True, verbose=2
    )
    final_val_accuracy = avg_val_accuracies[-1]
    results.append({'trial': i + 1, 'params': params, 'final_val_accuracy': final_val_accuracy})
    if final_val_accuracy > best_val_accuracy:
        best_val_accuracy = final_val_accuracy
        best_params = params

In [ ]:
df = pd.DataFrame([
    {
        'Trial': r['trial'],
        'learning_rate': r['params']['learning_rate'],
        'l2_lambda': r['params']['l2_lambda'],
        'batch_size': r['params']['batch_size'],
        'optimizer': r['params']['optimizer'],
        'val_accuracy': r['final_val_accuracy']
    }
    for r in results
]).sort_values(by='val_accuracy', ascending=False).reset_index(drop=True)

display(df)

df.to_csv('rs_results_summary.csv', index=False)
files.download('rs_results_summary.csv')

In [ ]:
with open("random_search_results.json", "w") as f:
    json.dump({
        "best_params": best_params,
        "best_val_accuracy": best_val_accuracy,
        "layer_sizes": layer_sizes
    }, f)

files.download("random_search_results.json")

In [ ]:
with open("random_search_results.json", "r") as f:
    data = json.load(f)
    best_params = data["best_params"]
    layer_sizes = data["layer_sizes"]
    best_val_accuracy = data["best_val_accuracy"]

learning_rate = best_params['learning_rate']
l2_lambda = best_params['l2_lambda']
batch_size = best_params['batch_size']
optimizer = best_params['optimizer']

In [ ]:
print("\n========================= K-Fold Cross-Validation with SGD (Augmented): ===========================")
avg_train_losses_sgd, avg_val_losses_sgd, avg_train_accuracies_sgd, avg_val_accuracies_sgd = k_fold_cross_validation(
    X_train, y_train_one_hot, layer_sizes, k=5, epochs=100, batch_size=batch_size, learning_rate=learning_rate, l2_lambda=l2_lambda, optimizer='sgd', augment=True,
)

In [ ]:
print("\nK-Fold Cross-Validation with Adam (Augmented):")
avg_train_losses_adam, avg_val_losses_adam, avg_train_accuracies_adam, avg_val_accuracies_adam = k_fold_cross_validation(
    X_train, y_train_one_hot, layer_sizes, k=5, epochs=100, batch_size=batch_size, learning_rate=learning_rate, l2_lambda=l2_lambda, optimizer='adam', augment=True
)

In [ ]:
# Ploting k-fold Cross-Validation Results (with SGD & Adam)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(avg_train_losses_sgd, label='Train Loss SGD')
ax1.plot(avg_val_losses_sgd, label='Val Loss SGD')
ax1.plot(avg_train_losses_adam, label='Train Loss Adam')
ax1.plot(avg_val_losses_adam, label='Val Loss Adam')
ax1.set_title('Average Loss (K-Fold)')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax2.plot(avg_train_accuracies_sgd, label='Train Acc SGD')
ax2.plot(avg_val_accuracies_sgd, label='Val Acc SGD')
ax2.plot(avg_train_accuracies_adam, label='Train Acc Adam')
ax2.plot(avg_val_accuracies_adam, label='Val Acc Adam')
ax2.set_title('Average Accuracy (K-Fold)')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()
plt.tight_layout()
plt.savefig('kfold_loss_accuracy_sgd_adam.png')
files.download('kfold_loss_accuracy_sgd_adam.png')
# plt.close()

In [ ]:
cm = confusion_matrix(np.argmax(y_test_one_hot, axis=1), y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Confusion Matrix (No Augmentation)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig('confusion_matrix_no_aug.png')
files.download('confusion_matrix_no_aug.png')
# plt.close()